# Creación y Almacenamiento de Embeddings en Chroma DB

En el presente Notebook se crearán la documentación base del esquema reducido con el que trabajaremos, basada en el `information_schema` correspondiente, que luego se nutrirá por fuera con mayor información relevante para negocio. 

Adicionalmente, por fuera del Notebook, se confeccionará documentación adicional, como reglas de negocio y few-shot examples para, finalmente, obtener chunks de todos estos documentos y crear y almacenar los mismos, con su metadata asociada, en una base de datos vectorial de Chroma DB.

## Inicialización

### Librerías



In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from collections import defaultdict, OrderedDict
import yaml
import json
import uuid
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_chroma import Chroma
import chromadb
from dotenv import load_dotenv

load_dotenv('../.env')
pd.set_option('display.max_columns', None)
yaml.add_representer(OrderedDict, lambda dumper, data: dumper.represent_dict(data.items()))

notebook_dir = os.getcwd() 
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)

from src.pg_sql import execute_query


### Constantes

In [2]:
DATABASE = 'database'
SCHEMAS = 'schemas'
TABLES = 'tables'
COLUMNS = 'columns'
DESCRIPTION = 'description'
NAME = 'name'
DATA_TYPE = 'data_type'
PRIMARY_KEY = 'is_primary_key'
FOREIGN_KEY = 'is_foreign_key'
REFERENCE = 'reference'
TO_DO = '[To be completed ...]'

OUTPUT_PATH = '../data/embeddings/auxs'
MDL_PATH = '../data/embeddings/documents/MDL_adventure_works_dw.yaml'

CHROMA_DB_PATH = '../data/embeddings/chroma'
TABLES_SUMMARY_COLLECTION_NAME = 'mdl_tables_summary'
COLUMNS_COLLECTION_NAME = 'mdl_columns'

AZURE_OPENAI_EMBEDDING_MODEL = 'text-embedding-3-small'

### Funciones aplicables a todos los documentos

In [3]:
def add_docs_to_chroma_col(chunks: list[dict], collection: Chroma) -> list[str]:
    """
    Store a list of document chunks into a ChromaDB collection using langchain.

    Each chunk is expected to be a dictionary containing 'content' (text of the chunk)
    and 'metadata' (associated metadata) keys. The function generates unique IDs and, if required,
    computes embeddings for each document chunk before adding them to the collection.

    Args:
        chunks (list[dict]): A list of dictionaries, each with keys:
                             - 'content' (str): The text content of the chunk.
                             - 'metadata' (dict): Metadata associated with the chunk.
        collection (langchain_chroma.Chroma): The target ChromaDB collection by langchain to store the data in.

    Returns:
        ids[list[str]]
    """

    ids = [str(uuid.uuid4()) for _ in chunks]

    collection.add_documents(
        ids = ids,
        documents= chunks
    )

## Fichero YAML con MDL del esquema de interés

### Obtener fichero base desde `information_schema`

Primero obtendremos un fichero base construido utilizando la query `/data/embeddings/auxs/get_information_schema.sql`, sobre el que luego se añadirá metadata extra. Para esto, definimos algunas funciones que nos serán de utilidad:

In [4]:
def get_information_schema(query_path: str, db_names_list: list[str], schema_names_list: list[str]) -> str:
    """
    Reads an SQL query from a file and replaces placeholder lists with formatted strings.

    This function is designed to work with SQL queries that have specific placeholders
    for database and schema names. It reads the query from the given file path, 
    formats the input lists of names into a single quoted, comma-separated string, 
    and replaces the placeholders in the query.

    Args:
        query_path (str): The file path to the SQL query. The query should
                          contain the placeholders `[db_names_list]` and
                          `[schema_names_list]`.
        db_names_list (list[str]): A list of database names to be formatted
                                   and inserted into the query.
        schema_names_list (list[str]): A list of schema names to be formatted
                                       and inserted into the query.

    Returns:
        str: The complete SQL query with the placeholders replaced by
             the formatted database and schema names.

    Raises:
        FileNotFoundError: If the specified query_path does not exist.
        
    Example:
        >>> from pathlib import Path
        >>> # Assume 'my_query.sql' contains:
        >>> # SELECT * FROM information_schema.tables WHERE table_schema IN ([schema_names_list])
        >>> # And we create a dummy file for the example:
        >>> Path('my_query.sql').write_text("SELECT * FROM information_schema.tables WHERE table_schema IN ([schema_names_list])")
        >>> db_list = ['db1', 'db2']
        >>> schema_list = ['schema_a', 'schema_b']
        >>> get_information_schema('my_query.sql', db_list, schema_list)
        "SELECT * FROM information_schema.tables WHERE table_schema IN ('schema_a', 'schema_b')"
    """
    query = Path(query_path).read_text()

    db_names = "'" + "', '".join(db_names_list) + "'"
    schema_names = "'" + "', '".join(schema_names_list) + "'"

    return query.replace('[db_names_list]', db_names).replace('[schema_names_list]', schema_names)



def format_yaml(yaml_str: str) -> str:
    """
    Formats a YAML string by adding a blank line before each list item
    that isn't preceded by a list key.
    """
    last_line = ''
    last_line_list_init = False
    last_line_empty = False

    lines = list()

    for line in yaml_str.split('\n'):
        if line.strip().startswith('-') and not last_line_list_init and not last_line_empty:
            last_line += '\n'

        lines.append(last_line)
        last_line = line
        last_line_list_init = last_line.strip().endswith(':')
        last_line_empty = last_line.strip()==''

    lines.append(line)

    return '\n'.join(lines)

In [5]:
GET_INFORMATION_SCHEMA_SQL = '../data/embeddings/auxs/get_information_schema.sql'
DB_NAME = 'adventure_works_dw'
SCHEMA_NAME = 'sales'


information_schema_data = execute_query(get_information_schema(
    query_path= GET_INFORMATION_SCHEMA_SQL,
    db_names_list= [DB_NAME],
    schema_names_list= [SCHEMA_NAME]
))

Veamos el aspecto que tienen los resultados de nuestra query:

In [6]:
information_schema_data

[{'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'customer_key',
  'column_type': 'INT4',
  'primary_key': True,
  'foreign_key': False,
  'target': None},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'geography_key',
  'column_type': 'INT4',
  'primary_key': False,
  'foreign_key': True,
  'target': 'sales.dim_geography.geography_key'},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'customer_full_name',
  'column_type': 'TEXT',
  'primary_key': False,
  'foreign_key': False,
  'target': None},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'birth_date',
  'column_type': 'DATE',
  'primary_key': False,
  'foreign_key': False,
  'target': None},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
 

Lo convertimos en un Data Frame de Pandas para que sea más vistoso:

In [7]:
pd.DataFrame(information_schema_data)

,db_name,schema_name,table_name,column_name,column_type,primary_key,foreign_key,target
0,adventure_works_dw,sales,dim_customer,customer_key,INT4,True,False,None
1,adventure_works_dw,sales,dim_customer,geography_key,INT4,False,True,sales.dim_geography.geography_key
2,adventure_works_dw,sales,dim_customer,customer_full_name,TEXT,False,False,None
3,adventure_works_dw,sales,dim_customer,birth_date,DATE,False,False,None
4,adventure_works_dw,sales,dim_customer,marital_status,BPCHAR(1),False,False,None
...,...,...,...,...,...,...,...,...
107,adventure_works_dw,sales,fact_sales,freight,NUMERIC,False,False,None
108,adventure_works_dw,sales,fact_sales,order_date,DATE,False,False,None
109,adventure_works_dw,sales,fact_sales,due_date,DATE,False,False,None
110,adventure_works_dw,sales,fact_sales,ship_date,DATE,False,False,None


Ahora procederemos a crear el YAML base con el MDL de nuestro esquema, que podremos tomar como punto de partida para luego añadirle metadata extra manualmente:

In [8]:
dbs_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for row in information_schema_data:
    db_name = row.get('db_name')
    schema_name = row.get('schema_name')
    table_name = row.get('table_name')
    
    dbs_data[db_name][schema_name][table_name].append(row)

for db_name, schemas_data in dbs_data.items():
    db = OrderedDict()
    db[DATABASE] = db_name
    db[DESCRIPTION] = TO_DO

    schemas = list()
    for schema_name, tables_data in schemas_data.items():
        schema = OrderedDict()
        schema[NAME] = schema_name
        schema[DESCRIPTION] = TO_DO

        tables = list()
        for table_name, columns_data in tables_data.items():
            table = OrderedDict()
            table[NAME] = table_name
            table[DESCRIPTION] = TO_DO

            columns = list()
            for column_data in columns_data:
                column = OrderedDict()
                column[NAME] = column_data.get('column_name')
                column[DESCRIPTION] = TO_DO
                column[DATA_TYPE] = column_data.get('column_type')
                
                if column_data.get('primary_key'):
                    column[PRIMARY_KEY] = True

                if column_data.get('foreign_key'):
                    column[FOREIGN_KEY] = True
                    column[REFERENCE] = column_data.get('target')

                columns.append(column)

            table[COLUMNS] = columns
            tables.append(table)
        
        schema[TABLES] = tables
        schemas.append(schema)
    
    db[SCHEMAS] = schemas

    mdl_file_path = f'{OUTPUT_PATH}/MDL_{db_name}.yaml'
    with open(mdl_file_path, 'w') as mdl:
        mdl.write(format_yaml(yaml.dump(db)))

    print(f'>  Fichero MDL base almacenado en {mdl_file_path}')

>  Fichero MDL base almacenado en ../data/embeddings/auxs/MDL_adventure_works_dw.yaml


### Creación de embeddings

Tomando como base el fichero obtenido en el apartado anterior, se ha creado un fichero que del MDL de nuestro esquema de interés que ha sido nutrido con metadata adicional, como descripciones para cada tabla y campo. Este fichero se encuentra en la ruta `/data/embeddings/documents/MDL_adventure_works_dw.yaml`.

Procederemos ahora a procesar este fichero para crear chunks de la información de cada una de las tablas y almacenar sus embeddings en nuestra base de datos vectorial de `chroma_db`, persistida en la ruta `/data/embeddings/chroma_db/`.

Procedemos ahora a definir funciones que nos permitirán chunkear el documento mdl desde un enfoque más integral a nivel de tablas, como también a un enfoque más granular a nivel de columnas:

In [9]:
def chunk_mdl_by_table_summary(mdl_data: dict) -> list[Document]:
    chunks = []

    database_name = mdl_data[DATABASE]

    for schema in mdl_data[SCHEMAS]:
        schema_name = schema[NAME]

        for table in schema[TABLES]:
            table_name = table[NAME]
            table_description = table[DESCRIPTION]
            table_columns = table[COLUMNS]
            
            chunk_content_list = [
                f'Database: {database_name}',
                f'Schema: {schema_name}',
                f'Table: {table_name}',
                f'Table description: {table_description}'
            ]

            table_primary_key = [col[NAME] for col in table_columns if col.get(PRIMARY_KEY)]
            if table_primary_key:
                chunk_content_list.append('Table PRIMARY KEY:')
                for pk in table_primary_key:
                    chunk_content_list.append(f'- {pk}')

            table_foreign_keys = [{'column_name': col[NAME], 'reference': col[REFERENCE]} for col in table_columns if col.get(FOREIGN_KEY)]
            if table_foreign_keys:
                chunk_content_list.append('Table FOREIGN KEYS (Column name, Reference):')
                for fk in table_foreign_keys:
                    chunk_content_list.append(f'- ({fk["column_name"]}, {fk["reference"]})')

            chunk_content = '\n'.join(chunk_content_list)

            metadata = {
                'database_name': database_name,
                'schema_name': schema_name,
                'table_name': table_name
            }

            chunks.append(Document(page_content= chunk_content, metadata= metadata))

    return chunks



def chunk_mdl_by_column(mdl_data: dict) -> list[Document]:
    chunks = []

    database_name = mdl_data[DATABASE]

    for schema in mdl_data[SCHEMAS]:
        schema_name = schema[NAME]

        for table in schema[TABLES]:
            table_name = table[NAME]
            table_columns = table[COLUMNS]

            for col in table_columns:
                chunk_content_list = list()

                if col.get(PRIMARY_KEY) or col.get(FOREIGN_KEY):
                    continue

                column_name = col[NAME]
                column_data_type = col[DATA_TYPE]
                column_description = col[DESCRIPTION]

                chunk_content_list.append(f'Column name: {column_name}')
                chunk_content_list.append(f'Column data type: {column_data_type}')
                chunk_content_list.append(f'Column description: {column_description}')

                chunk_content = '\n'.join(chunk_content_list)

                metadata = {
                    'database_name': database_name,
                    'schema_name': schema_name,
                    'table_name': table_name,
                    'column_name': column_name,
                    'column_data_type': column_data_type
                }

                chunks.append(Document(page_content= chunk_content, metadata= metadata))

    return chunks


Veamos ahora el aspecto que tiene, por ejemplo, el chunk de mayor tamaño para cada uno de los tipos:

In [10]:
with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)

chunks = chunk_mdl_by_table_summary(mdl_data)

max_len = max([len(c.page_content) for c in chunks])
print(f'{max_len=}')
print()
print([c.page_content for c in chunks if len(c.page_content) == max_len][0])
print()
print([c.metadata for c in chunks if len(c.page_content) == max_len][0])

max_len=1020

Database: adventure_works_dw
Schema: sales
Table: fact_sales
Table description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.
Table PRIMARY KEY:
- sales_order_number
- sales_order_line_number
Table FOREIGN KEYS (Column name, Reference):
- (product_key, sales.dim_product.product_key)
- (reseller_key, sales.dim_reseller.reseller_key)
- (employee_key, sales.dim_sales_person.employee_key)
- (customer_key, sales.dim_customer.customer_key)
- (promotion_key, sales.dim_promotion.promotion_key)
- (sales_territory_key, sales.dim_sa

In [11]:
with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)

chunks = chunk_mdl_by_column(mdl_data)

max_len = max([len(c.page_content) for c in chunks])
print(f'{max_len=}')
print()
print([c.page_content for c in chunks if len(c.page_content) == max_len][0])
print()
print([c.metadata for c in chunks if len(c.page_content) == max_len][0])

max_len=265

Column name: sale_source
Column data type: TEXT
Column description: Indicador de la fuente por la que ha sido realizado el pedido. reseller_sales=Pedido realizado por un vendedor para una tienda/distribuidor, internet_sales=Pedido realizado en línea por un cliente.

{'database_name': 'adventure_works_dw', 'schema_name': 'sales', 'table_name': 'fact_sales', 'column_name': 'sale_source', 'column_data_type': 'TEXT'}


Ahora procedemos a crear ambas colecciones de Chroma DB y a almacenar los embeddings de los chunks obtenidos en cada una:

In [ ]:
chroma_client = chromadb.PersistentClient(path= CHROMA_DB_PATH)
embeddings = AzureOpenAIEmbeddings(model=AZURE_OPENAI_EMBEDDING_MODEL)


with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)



table_summary_chunks = chunk_mdl_by_table_summary(mdl_data)
if table_summary_chunks:
    try:
        chroma_client.delete_collection(TABLES_SUMMARY_COLLECTION_NAME)
    except:
        pass

    table_summary_collection = Chroma(
        client= chroma_client,
        collection_name= TABLES_SUMMARY_COLLECTION_NAME,
        embedding_function= embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(table_summary_chunks, table_summary_collection)


column_reduced_chunks = chunk_mdl_by_column(mdl_data)
if column_reduced_chunks:
    try:
        chroma_client.delete_collection(COLUMNS_COLLECTION_NAME)
    except:
        pass

    column_collection = Chroma(
        client= chroma_client,
        collection_name= COLUMNS_COLLECTION_NAME,
        embedding_function= embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(column_reduced_chunks, column_collection)


Lanzamos ahora una consulta a cada una de las colecciones creadas:

In [27]:
chroma_client = chromadb.PersistentClient(path= CHROMA_DB_PATH)
embeddings = AzureOpenAIEmbeddings(model=AZURE_OPENAI_EMBEDDING_MODEL)

table_summary_collection = Chroma(
    client= chroma_client,
    collection_name= TABLES_SUMMARY_COLLECTION_NAME,
    embedding_function= embeddings
)

column_collection = Chroma(
    client= chroma_client,
    collection_name= COLUMNS_COLLECTION_NAME,
    embedding_function= embeddings
)

In [28]:
QUERY = 'Los 10 artículos más comprados.'

In [31]:
table_summary_results = table_summary_collection.similarity_search_with_relevance_scores(
    query= QUERY,
    k= 10,
    score_threshold= 0.20
)

relevant_columns = []
for table_result in table_summary_results:
    database_name = table_result[0].metadata['database_name']
    schema_name = table_result[0].metadata['schema_name']
    table_name = table_result[0].metadata['table_name']

    table_filter = {
        "$and": [
            {"database_name": {"$eq": database_name}},
            {"schema_name": {"$eq": schema_name}},
            {"table_name": {"$eq": table_name}}
        ]
    }

    column_reduced_results = column_collection.similarity_search_with_relevance_scores(
        query= QUERY,
        k= 15,
        filter=table_filter,
        score_threshold= 0.15
    )

    if column_reduced_results:
        relevant_columns.append({
            'table_summary': {
                'content': table_result[0].page_content,
                'relevance_score': table_result[1]
            },
            'columns': [{'content': col[0].page_content, 'relevance_score': col[1]} for col in column_reduced_results]
        })

if not relevant_columns:
    relevant_columns = [f'No docs retrieved for the query "{QUERY}"']

print(json.dumps(relevant_columns, indent=2, ensure_ascii=False))

No relevant docs were retrieved using the relevance score threshold 0.15


[
  {
    "table_summary": {
      "content": "Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promot

In [20]:
print(relevant_columns)

[{'table_summary': {'content': 'Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promotion_key)\n- (sa

In [21]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0)

summarizer_prompt = ChatPromptTemplate.from_template('''
Eres un asistente experto en bases de datos y SQL.
Tu tarea es generar un resumen conciso y relevante de la información dada sobre tablas y columnas,
teniendo en cuenta la pregunta del usuario.

# Pregunta del usuario:
{user_query}

# Datos recuperados:
{retrieved_docs}

Instrucciones para el resumen:
- Incluye solo las tablas y columnas necesarias para responder la pregunta.
- Mantén descripciones útiles para entender qué almacena cada columna.
- Conserva información clave sobre relaciones entre tablas si es necesaria.
- Elimina cualquier detalle irrelevante.
- Sé claro y breve para que otro agente pueda usar este resumen para crear una consulta SQL.
- Responde en español.
''')

summarizer_chain = summarizer_prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieved_docs': json.dumps(relevant_columns, ensure_ascii=False)
})

print(contexto_resumido)

Para obtener los 10 artículos más comprados, se utilizarán las siguientes tablas y columnas de la base de datos `adventure_works_dw`:

### Tablas relevantes:

1. **Tabla: fact_sales**
   - **Descripción**: Contiene el detalle de las órdenes de ventas entregadas, con información a nivel de línea.
   - **Columnas relevantes**:
     - **order_quantity** (INT2): Cantidad vendida de cada producto.
     - **product_key** (FK): Clave del producto, que se relaciona con la tabla `dim_product`.

2. **Tabla: dim_product**
   - **Descripción**: Almacena información detallada de cada producto vendido.
   - **Columnas relevantes**:
     - **product_key** (PK): Clave del producto.
     - **english_product_name** (VARCHAR(50)): Nombre del producto en inglés.
     - **spanish_product_name** (VARCHAR(50)): Nombre del producto en español.

### Relación entre tablas:
- La columna `product_key` en `fact_sales` es una clave foránea que se relaciona con la columna `product_key` en `dim_product`, permitiendo 

In [22]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0)

SYSTEM_PROMPT = """
You are a context summarizer that receives retrieval documents in JSON format.
Your role is to:

1. Analyze the given JSON documents carefully.
2. Identify and extract only the information relevant to answer the user's question.
3. Summarize relevant tables, columns, keys, and descriptions into a clear and concise text.
4. Exclude any irrelevant or low-relevance details.
5. Ensure factual accuracy; do not fabricate information.
6. Do NOT generate or suggest any SQL queries.
7. Match the language of the summary to the user's query.

Input:
- Retrieval documents (JSON): {retrieval_documents}

Output:
- A concise summary of relevant context—focused on tables, columns, keys, and descriptions—that will help generate an accurate SQL query.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{user_query}")
])

summarizer_chain = summarizer_prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieved_docs': json.dumps(relevant_columns, ensure_ascii=False)
})

print(contexto_resumido)

Para obtener los 10 artículos más comprados, se utilizarán las siguientes tablas y columnas de la base de datos `adventure_works_dw`:

### Tablas relevantes:

1. **Tabla: fact_sales**
   - **Descripción**: Contiene el detalle de las órdenes de ventas entregadas, con información a nivel de línea.
   - **Columnas relevantes**:
     - **order_quantity** (INT2): Cantidad vendida de cada producto.
     - **product_key** (FK): Clave del producto, que se relaciona con la tabla `dim_product`.

2. **Tabla: dim_product**
   - **Descripción**: Almacena información detallada de cada producto vendido.
   - **Columnas relevantes**:
     - **product_key** (PK): Clave del producto.
     - **english_product_name** (VARCHAR(50)): Nombre del producto en inglés.
     - **spanish_product_name** (VARCHAR(50)): Nombre del producto en español (si está disponible).

### Relación entre tablas:
- La columna `product_key` en `fact_sales` es una clave foránea que se relaciona con la columna `product_key` en `dim_p